# L5: Information extraction

In [1]:
# import all the packages needed
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
import spacy
from IPython.display import Image
#from collections import Counter

In [2]:
# import for problem 5
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

Information extraction (IE) is the task of **identifying named entities and semantic relations between these entities** in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

We start by loading spaCy:

In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [4]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(" "))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main **data set** for this lab is a **collection of news wire articles** in which **mentions of named entities** have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [5]:
import bz2
import csv
import pandas as pd

with bz2.open("ner-train.tsv.bz2", 'rt') as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open("ner-dev.tsv.bz2", 'rt') as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a **Wikipedia page name** or the **generic label** `--NME--`

The following cell prints the first five samples from the training data:

In [6]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

In [7]:
# print the whole sentence with the id 0000-000
df_train.loc[0]["sentence"]

'EU rejects German call to boycott British lamb .'

## Problem 1: Evaluation measures

To warm up, we ask you to write code to **print** the **three measures** that you will be using **for evaluation**:

In [8]:
Image(filename = "precisionANDrecall.png", width=600, height=600)

FileNotFoundError: [Errno 2] No such file or directory: 'precisionANDrecall.png'

In [ ]:
Image(filename = "F1score.png", width=600, height=600)

In [ ]:
def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.
    
    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    # TODO: Replace the next line with your own code
    
    # Compute precision
    precision = len(gold.intersection(pred))/len(pred) * 100 # * 100 for percentage
    # Compute recall
    recall = len(pred.intersection(gold))/len(gold) * 100 # * 100 for percentage
    # Compute F1 score
    F1 = 2 * (precision * recall) / (precision + recall)
    
    print("precision score: {}%".format(round(precision,2)))
    print("recall score: {}%".format(round(recall,2)))
    print("F1 score: {}%".format(round(F1,2)))
    

To test your code, you can run the following cell:

In [ ]:
evaluation_report(set(range(3)), set(range(5)))

This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the **first tasks** that an information extraction system has to solve is to **locate and classify (mentions of) named entities**, such as *persons and organizations*. Here we will tackle the simpler task of **recognizing spans of tokens** that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to **evaluate the performance** of this component when **predicting entity spans** in the development data.

Start by implementing a generator function that ***yields* the gold-standard spans** in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [ ]:
df_dev.head()

In [ ]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    
    # TODO: Replace the next line with your own code
    for row in df_dev.itertuples():
        yield row.sentence_id, row.beg, row.end
    

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a **total** of **5,917** unique triples. The **first triple and the last triple** should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

**Total of unique triples**

In [ ]:
# check the total length

# use the gold_spans function and save as set
spans_dev_gold = set(gold_spans(df_dev))
print("Total of {} unique triples.".format(len(spans_dev_gold)))

**first triple and the last triple**

In [ ]:
# check the order of the triples
# run a for loop and print the 1 and the last element
for i, item in enumerate(spans_dev_gold):
    if i == 0:
        print(item)
    if i == (len(spans_dev_gold)-1):
        print(item)

** This is not the same order as expected. **

Your next task is to write code that **calls spaCy to predict the named entities in the development data**, and to **evaluate the accuracy of these predictions** in terms of precision, recall, and F1. **Print** these **scores** using the function that you wrote for Problem&nbsp;1.

In [ ]:
# TODO: Write code here to run and evaluate the spaCy NER on the development data

In [ ]:
def prob2_func(df):
    # want to make predictions
    # therefore we want to use spacys named Entity Recognition 
    # this we do by nlp() of a sentence, 
    # we need to access each sentence of the df (df_dev)
    
    # loop over each sentence by its index
    for sentence_index in df.sentence_id.unique():
        
        # match the index of the loop with the index of the data frame and take its sentence
        # - helpful link, how to work with iloc & loc - link below
        #print(df.loc[df.sentence_id == sentence_index, "sentence"])
        sentence = df.loc[df.sentence_id == sentence_index, "sentence"]
        sentence = sentence.values[0] # 0 - access the sentence
    
        # named Entity Recognition, we want the start and the end of entity 
        doc = nlp(sentence) 
        for ent in doc.ents:
            #print(sentence_index, ent.start, ent.end)
            yield sentence_index, ent.start, ent.end

In [ ]:
# make prediction for problem 2 & save as set
pred_prob2 = set(prob2_func(df_dev))

In [ ]:
evaluation_report(spans_dev_gold, pred_prob2)

## Problem 3: Error analysis

As you were able to see in **Problem&nbsp;2**, the **span accuracy** of the named entity recognizer is **far from perfect**. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to **analyse** this **result in more detail**.

**Write code** that **prints the false positives and the false negatives** from the **automatic prediction**. Have a look at the output. What are your **observations**? How could you **improve the result**? **Discuss** these questions in a short text.

In [ ]:
# TODO: Write code here to do your analysis

** Print false positives and the false negatives of the prediction**

In [ ]:
Image(filename = "FalsePositive_FalseNegative.png", width=600, height=600)

In [ ]:
def error_analysis(gold, pred, nr_print, False_Positive = True):
    if False_Positive == True:
        # ---- Case: false positive
        print("Case: false positive")
        difference = pred.difference(gold)
    else:
        # ---- Case: false negative
        print("Case: false negative")
        difference = gold.difference(pred)
        
    # init the values I want to print later on
    difference_list = []
    difference_list_entity = []
    difference_list_both = []

    for item in difference:
        #print(item)
        # take the sentence of the sentence_id, can have multiple rows with the same sentence
        sentence = df_dev.loc[df_dev.sentence_id == item[0], ["sentence"]]
        # therefore, take the first value of the dataframe 
        sentence = sentence.iloc[0]
        # converte to string and split to access specific word
        sentence = sentence.values[0].split()
        word = sentence[item[1]:item[2]]
        # save specific word of specific item
        #difference_list.append(word)

        #------ get entitiy
        # converte list to string
        word_string = ' '.join(map(str, word))
        # get the entitiy lable of the object
        doc = nlp(word_string)
        for ent in doc.ents:
            difference_list_both.append([word_string,ent.label_])
            difference_list_entity.append(ent.label_)

    # ------------------- print the results -------------------
    # print the word with its entity
    print("The first {} word entitiy pairs:".format(nr_print))
    print(difference_list_both[0:nr_print])

    # print a count of the entities
    entity_counts = pd.Series(difference_list_entity)
    all_counts = len(entity_counts)
    entity_counts = entity_counts.value_counts()
    print("Entity types count:")
    print(entity_counts)
    print("Sum: {}".format(all_counts))

In [ ]:
# error analysis - case: false positive
error_analysis(spans_dev_gold,pred_prob2,10, False_Positive = True)

In [ ]:
# error analysis - case: false negative
error_analysis(spans_dev_gold,pred_prob2,10, False_Positive = False)

** Interpretation **

- **Case: false positive**

It can be seen that especially often numbers like the category Cardinal & date were wrongly classified.
These are by far the most common mistakes. 

- **Case: false negative**

Here it can be seen that organizations and individuals are generated the most misclassifications, as well as countries. 

However, these have a similar number of errors to the false positive case. Overall, the false positive case has a much higher number of errors than the false negative case. 

To improve the score, a similar procedure as in problem 3 could be performed, but without the entity types with the most errors.


Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to **write code to pre-process the data and/or post-process the output produced by spaCy**. You should be able to **improve** the **F1 score from Problem&nbsp;2 by at last 15 percentage points**.

In [ ]:
# TODO: Write code here to improve the span prediction from Problem 2

In [ ]:
# code as in prob2_func + light modifications
def prob3_func(df):
        for indx in df.sentence_id.unique():
            sentence = df.loc[df.sentence_id==indx,'sentence']
            doc = nlp(sentence.values[0])
            for ent in doc.ents:
                # modification at this part
                # exclude some specific Named Entity Recognition
                # ---- Test combinations
                #if ent.label_ not in ['CARDINAL', 'DATE', 'ORG', 'Person', 'GPE']:
                #if ent.label_ not in ['CARDINAL', 'DATE', 'ORG', 'PERSON', 'ORDINAL', 'GPE', 'TIME', 'QUANTITY','PERCENT','MONEY','NORP', 'EVENT']:
                # ----
                if ent.label_ not in ['CARDINAL', 'ORDINAL', 'QUANTITY', 'MONEY', 'PERCENT', 'TIME', 'DATE']:
                    yield indx,ent.start,ent.end
# make predictions on the same dataset with the new function                   
new_preds = set(prob3_func(df_dev))

**Show** that you achieve the **performance goal** by reporting the evaluation measures that you implemented in Problem&nbsp;1.

In [ ]:
evaluation_report(spans_dev_gold,new_preds)

In [ ]:
print("The F1 score improved by {} %.".format(round(75.89-59.9,3)))

Before going on, we ask you to **store** the **outputs of the improved named entity recognizer on the development data in a new data frame**. This **new frame** should have the **same layout as the original data frame for the development data** that you loaded above, but should contain the predicted start and end positions for each token span. As the `label` of **each span**, you can use the **special value** `--NME--`.

In [ ]:
# TODO: Write code here to store the predicted spans in a new data frame

In [ ]:
# save the prediction as list
preds_list = list(new_preds)

x=[item[0] for item in preds_list]
y=[item[1] for item in preds_list]
z=[item[2] for item in preds_list]

# create a list with the indexes for each predicted span
indexes_list = []
for i in range(len(x)):
    indexes_list.append(df_dev.index[(df_dev['sentence_id']==x[i]) & (df_dev['beg']==y[i]) & (df_dev['end']==z[i])].tolist())
    

In [ ]:
from functools import reduce
# flatten the indexes list 
flat_list = reduce(lambda x, y: x+y, indexes_list)

In [ ]:
# create the dataframe
df_dev_small = df_dev.iloc[flat_list,:]
df_dev_small.drop(['label'],axis=1)
df_dev_small.label='--NME--'
df_dev_small.head(n=10)

## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, **for each span we want to predict the Wikipedia page that this mention references**.

Start by **extending** the generator function that you implemented in **Problem&nbsp;2** to labelled spans.

In [ ]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    
    # TODO: Replace the next line with your own code
    # code from Probem 2 - gold_spans
    for row in df_dev.itertuples():
        # added lables
        yield row.sentence_id, row.beg, row.end, row.label 

A **naive baseline for entity linking** on our data set is to **link each mention span to the Wikipedia page name** that we get when we ***join* the tokens in the span by underscores**, as is standard in Wikipedia page names. Suppose, **for example**, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

**Implement** this **naive baseline and evaluate its performance**. Print the evaluation measures that you implemented in Problem&nbsp;1.

<div class="alert alert-warning">
    Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.
</div>

In [ ]:
# TODO: Write code here to implement the baseline

In [ ]:
df_dev_small.head(10)

In [ ]:
# function to create predictions
def prob4_func(df):
    # run every row of the df
    for row in range(df_dev_small.shape[0]):
        # get the sentence_id 
        sentence_id = df_dev_small.iloc[row,0]
        
        # get the beg and the end of specific word
        beg = df_dev_small.iloc[row,2]
        end = df_dev_small.iloc[row,3]
        
        # access every word of the sentence
        sentence = df_dev_small.iloc[row,1].split()
        # filter for word(s) we want to have
        word = sentence[beg:end]
        
        # join the words with "_"
        words = "_".join(word)
        
        yield sentence_id, beg, end, words
# make predictions 
preds4 = set(prob4_func(df_dev_small))

In [ ]:
# use the gold_mentions function and save as set
gold_mentions_set = set(gold_mentions(df_dev_small))

In [ ]:
# evaluation
evaluation_report(gold_mentions_set,preds4)

## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to **entity linking exploit information in knowledge bases**. In our case, where **Wikipedia is the knowledge base**, one particularly useful type of information are links to other Wikipedia pages. In particular, we can **interpret the anchor texts** (the highlighted texts that you click on when you click on a link) as **mentions of the entities** (pages) that they link to. This allows us to harvest long lists over mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [ ]:
with bz2.open("kb.tsv.bz2", 'rt') as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [ ]:
df_kb.loc[df_kb["mention"] == "Sweden"]

As you can see, **each row** of the data frame **contains a pair $(m, e)$ of a mention $m$ and an entity $e$**, as well as the **conditional probability $P(e|m)$ for mention $m$ referring to entity $e$**. These probabilities were **estimated based on the frequencies of mention–entity pairs in the knowledge base**. The **example** shows that the **anchor text &lsquo;Sweden&rsquo;** is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the **most probable pairing stands first**.

**Implement** an **entity linking method** that **resolves each mention to the most probable entity in the data frame**. If the mention is not included in the data frame, you can predict the generic label `--NME--`. **Print the precision, recall, and F1** of your method using the function that you implemented for Problem&nbsp;1.

In [ ]:
# TODO: Write code here to implement the "most probable entity" method.

In [ ]:
df_kb.head()

In [ ]:
# function to create predictions
def prob5_func(df):
    # run every row of the df
    for row in range(df.shape[0]):
        # get the mention 
        mention = df.iloc[row,0]
        
        # get entity with highest prob
        mention_df = df.loc[df["mention"] == mention]
        
        ### ------ flag ------
        # if df not empty -> take highest prob
        # else -- NME --
        if mention_df.shape[0] != 0:
            mention_df_highest_entity = mention_df.iloc[0,1]
            yield mention, mention_df_highest_entity
        else:
            yield mention, "--NME--" 
# make predictions
preds5 = set(prob5_func(df_kb))
#print(preds5)

In [ ]:
# We also need a gold-standard to evaluate the results
def gold_mention_entitiy(df):
    # TODO: Replace the next line with your own code
    # code from Probem 2 - gold_spans
    for row in df.itertuples():
        # added lables
        yield row.mention, row.entity

# create gold standard
gold_mention_entity_set = set(gold_mention_entitiy(df_kb))
#print(gold_mention_entity_set)

In [ ]:
# evaluation
evaluation_report(gold_mention_entity_set,preds5)

<strong>Question 1 - Problem 5: Results accuracy </strong> 

> Not sure if I did understand the question 100% correctly.
The accuracy seems strange to me, especially since I read problem 6. Not sure if the results should be that good.

## Problem 6: Context-sensitive disambiguation

Consider the **entity mention &lsquo;Lincoln&rsquo;**. The **most probable entity** for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be **better off to predict** [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should **try to disambiguate between different entity references based on the textual context on the page from which the mention was taken**. Your task in this last problem is to implement this idea.

**Set up** a **dictionary** that **contains**, for each **mention $m$ that can refer to more than one entity $e$**, a **separate Naive Bayes classifier to predict the correct entity $e$**, given the textual context of the mention. As the prior probabilities of the classifier, **choose** the **probabilities $P(e|m)$** that you used in Problem&nbsp;5. To let you **estimate the context-specific probabilities**, we have compiled a data set with mention contexts:

In [ ]:
with bz2.open("contexts.tsv.bz2") as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This **data frame contains**, for each **ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer**, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is **defined as a bag of words** containing the mention itself, as well as the **5 tokens to the left and the 5 tokens to the right** of the mention. Here are a few examples:

In [ ]:
df_contexts.head()

From this data frame, it is easy to select the **data that you need to train the classifiers** – the **contexts** and corresponding **entities for all mentions**. To illustrate this, the following cell **shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;**:

In [ ]:
df_contexts.context[df_contexts.mention == "Lincoln"]

**Implement the context-sensitive disambiguation method and evaluate its performance**. Here are some more hints that may help you along the way:

**Hint 1:** The **prior probabilities** for a Naive Bayes classifier can be **specified using** the `class_prior` option. You will have to provide the **probabilities** in the same **order as the alphabetically sorted class (entity) names.**

**Hint 2:** To **tune** the **performance of your method**, you can try to **tweak the behaviour of the vectorizer** (for example, should it apply lowercasing or not?) and the **width of the window** from which you are extracting context tokens at prediction time.

**Hint 3:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only **one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label**.

In [ ]:
# TODO: Write code here to implement the context-sensitive disambiguation method

**Test - Start**

This part can be ignored

- ** Test for piror probs & fit **

In [ ]:
df_contexts.mention

In [ ]:
print(len(df_contexts.mention))
print(len(df_contexts.mention.unique()))
print("There are {} mention which are more often in the df.".format(len(df_contexts.mention) - len(df_contexts.mention.unique())))

In [ ]:
# each mention m, just take the unique mentions
for i in df_contexts.mention.unique():
    print(i)

In [ ]:
df_contexts.entity

In [ ]:
# dict test
dict = {
    "Phillip": ["A", "B"],
    "Marie": "C"
}

In [ ]:
print(dict)

In [ ]:
dict["Phillip"][1]

In [ ]:
# fill dict
dict["Rosa"] = ["D", "E"]

In [ ]:
dict

In [ ]:
dict["Phillip"][0]

In [ ]:
# fill a dict in a loop, generate test data set for this
test_df = df_contexts.head()

In [ ]:
test_df

In [ ]:
test_df.entity[test_df.mention == "1970"].tolist()

In [ ]:
test_dict = {}
for mention in test_df.mention.unique():
    test_dict[mention] = test_df.entity[test_df.mention == mention].tolist()

In [ ]:
test_dict

In [ ]:
len(test_dict)
# number of mentions

In [ ]:
# create a list of each mention and all correspondingly entities
dict_m_e = {}
for mention in df_contexts.mention.unique():
    dict_m_e[mention] = df_contexts.entity[df_contexts.mention == mention].tolist()

In [ ]:
# check the length of mentions, should be 1245 (tested before)
len(dict_m_e)

In [ ]:
print(dict_m_e)

In [ ]:
df_kb.head()

In [ ]:
# check the probs from the mention test above
df_kb.loc[df_kb.mention == "1970"]

In [ ]:
df_test = df_kb.loc[df_kb.mention == "1990 World Cup"]
df_test

In [ ]:
df_test.sort_values(by = ["entity"]).prob

In [ ]:
# same order as the alphabetically sorted class (entity) names.
df_test.sort_values(by = ["entity"]).prob.tolist()

In [ ]:
for mention in df_contexts.mention.unique():
    class_prob_df = df_kb.loc[df_kb.mention == mention]
    class_prob_test = class_prob_df.sort_values(by = ["entity"]).prob.tolist()

In [ ]:
class_prob_test
# saves the prob for each mention
# run the pipline in loop & input this vector for each iteration!!!

In [ ]:
# pipline test
Pipeline_CV_MNB_test = Pipeline([
    ('vect', CountVectorizer(lowercase=False)), # default = True
    ('clf', MultinomialNB(class_prior = class_prob_test)),
])

In [ ]:
# fit pipeline with X (mention) & y (entity)
pipefit_test = Pipeline_CV_MNB_test.fit(X = df_contexts.context[df_contexts.mention == mention],
                                   y = df_contexts.entity[df_contexts.mention == mention]) 

In [ ]:
# pipeline
pipefit_test

- **Test for prediction**

In [ ]:
test_context = df_contexts[df_contexts.mention == mention]
test_context

In [ ]:
test_context = test_context.context.tolist()[0]
print(test_context)
print(type(test_context))

In [ ]:
pipefit_test.predict([test_context])
# nice that looks good!!!

In [ ]:
pipefit_test.predict([test_context])[0]

In [ ]:
df_dev_small.tail(10)

In [ ]:
df_dev_small.shape

In [ ]:
df_dev_small.shape[0]

In [ ]:
# want to make predictions from df of problem 3
# we gonna label the --NME-- of df_dev_small

# we have to access the mention for each sentence in the df_dev_small
row = df_dev_small.shape[0] -1
sentence_test = df_dev_small.iloc[row,1].split()
print(sentence_test)
beg = df_dev_small.iloc[row,2]
end = df_dev_small.iloc[row,3]
mention_test = sentence_test[beg:end]
print(mention_test)

In [ ]:
# we also need to access the context, which is needed to make the prediction of the mention
# 5 tokens to the left and the 5 tokens to the right of the mention
context_test = sentence_test[max(0,(beg-5)):min((end+5), len(sentence_test))]
context_test

In [ ]:
" ".join(context_test)

In [ ]:
min(9,len(context_test))

In [ ]:
print(" ".join(mention_test))

In [ ]:
# df to check, later check if mention exists
mention_df_test = df_contexts[df_contexts.mention == " ".join(mention_test)]
mention_df_test

In [ ]:
# show that the data is empty, for the frame
mention_df_test.shape[0]
# if it´s 0, I can´t make any prediction!!

In [ ]:
mention_test_string = " ".join(mention_test)
context_test_string = " ".join(context_test)
dict_fit[mention_test_string].predict([context_test_string])

In [ ]:
dict_fit[mention_test_string].predict([context_test_string])[0]

** Test - End**

In [ ]:
# separate Naive Bayes classifier to predict the correct entity
# given the textual context of the mention

# init the dictionary
dict_fit = {}

for mention in df_contexts.mention.unique():
    
    # set up the pipeline

    # we have to the input for MultinomialNB(class_prior= input)
    # use the prob from problem 5

    # get class_probs
    class_prob_df = df_kb.loc[df_kb.mention == mention]
    class_prob = class_prob_df.sort_values(by = ["entity"]).prob.tolist() # same order as the alphabetically sorted class (entity) names.

    # create pipeline
    Pipeline_CV_MNB = Pipeline([
        ('vect', CountVectorizer(lowercase=False)), # default = True
        ('clf', MultinomialNB(class_prior = class_prob)),
    ])
    
    # fit every mention
    # fit the pipeline with X (mention) & y (entity)
    fit = Pipeline_CV_MNB.fit(X = df_contexts.context[df_contexts.mention == mention],
                              y = df_contexts.entity[df_contexts.mention == mention])
    
    dict_fit[mention] = fit

In [ ]:
# function to create predictions
def prob6_func(df):
    # run every row of the df
    for row in range(df.shape[0]):
        # want to make predictions from df of problem 3
        # we gonna label the --NME-- of df_dev_small

        # we have to access the mention for each sentence in the df_dev_small
        sentence = df.iloc[row,1].split()

        # we need the beginning and the end to access the mention
        beg = df.iloc[row,2]
        end = df.iloc[row,3]
        mention = sentence[beg:end]

        # we also need to access the context, which is needed to make the prediction of the mention
        # 5 tokens to the left and the 5 tokens to the right of the mention
        context = sentence_test[max(0,(beg-5)):min((end+5), len(sentence))]
        # need the context as string for the prediction
        context_string = " ".join(context)

        ### ------ flag ------

        # create df, later check if mention exists
        mention_test_string = " ".join(mention)
        mention_df = df_contexts[df_contexts.mention == mention_test_string]

        # if df not empty -> take highest prob
        # else -- NME --
        if mention_df.shape[0] != 0:
            # make predictions
            label = dict_fit[mention_test_string].predict([context_string])[0]
            yield mention_test_string, label
        else:
            yield mention_test_string, "--NME--"

In [ ]:
# make predictions
preds6 = set(prob6_func(df_dev_small))

In [ ]:
# evaluation
evaluation_report(gold_mention_entity_set,preds6)

You should **expect** to see a **small (around 1&nbsp;unit) increase** in both precision, recall, and F1. Published systems report a larger impact of context-sensitive disambiguation. Feel free to think about what could explain the relatively minor impact that we see here!

**Interpretation of the results**

<strong>Question 2 - Problem 6: Results accuracy </strong> 

> In problem 5 I had the problem that the accuracy of my opinion was very high, too high to be correct! 
In problem 6 I have the opposite problem. The accuracy is so bad that I can hardly imagine that this is correct.

**This was the last lab in the Text Mining course. Well done!**

<div class="alert alert-info">
    Please read the section ‘General information’ on the ‘Labs’ page of the course website before submitting this notebook!
</div>

**Helpful links**

- Problem 1

https://www.geeksforgeeks.org/intersection-function-python/

- Problem 2

https://www.geeksforgeeks.org/use-yield-keyword-instead-return-keyword-python/

https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/

- Problem 3

https://dev.to/svinci/intersection-union-and-difference-of-sets-in-python-4gkn

- Problem 6

https://www.w3schools.com/python/python_dictionaries.asp

https://stackoverflow.com/questions/30280856/populating-a-dictionary-using-for-loops-python

https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html